<h1>SMARCB1: Kosho et al (2014)</h1>
<p>We will process <a href="https://pubmed.ncbi.nlm.nih.gov/25168959/" target="__blank">Kosho, et al. (2014) Genotype-phenotype correlation of Coffin-Siris syndrome caused by mutations in SMARCB1, SMARCA4, SMARCE1, and ARID1A</a></p>
<p>pyphetools provides a convenient way of extracting HPO terms from typical tables presented in supplemental material. Typical tables can have the individuals in columns or rows.</p>
<p>The data are extracted from the SMARCB1 section of Table 1.</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import os
import sys
import numpy as np
from pyphetools.creation import *
from pyphetools.visualization import *
import pyphetools
print(f"pyphetools version {pyphetools.__version__}")

pyphetools version 0.6.5


<h2>Importing HPO data</h2>

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
PMID = "PMID:25168959"
title = "Genotype-phenotype correlation of Coffin-Siris syndrome caused by mutations in SMARCB1, SMARCA4, SMARCE1, and ARID1A"
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155", pmid=PMID, pubmed_title=title)
metadata.default_versions_with_hpo(version=hpo_version)

<h2>Importing the supplemental table</h2>
<p>The Table of the paper was copied into an Excel file that is included in the data subfolder</p>
<p>Here, we use the pandas library to import this file (note that the Python package called openpyxl must be installed to read Excel files with pandas, although the library does not need to be imported in this notebook). pyphetools expects a pandas DataFrame as input, and users can choose any input format available for pandas include CSV, TSV, and Excel, or can use any other method to transform their input data into a Pandas DataFrame before using pyphetools.</p>

In [3]:
df = pd.read_excel('input/PMID_25168959.xlsx')

In [4]:
df

,"Patient ID""",Nucleotide change,Amino acid change,Feeding difficulties,Nasal bridge,Phitrum,Upper lip vermilion,Thick lower lip vermilion,High palate,Cleft palate,...,Microcephaly,Sparse scalp hair,hypertrichosis,Thick eyebrows,Long eyelashes,Ptosis,Short 5th finger,Short 5th toe,Prominent interphalangeal joints,Prominent distal phalanges
0,L43,c.1089G>T,p.Lys363Asn,Yes,Narrow,Normal,Thin,No,Yes,No,...,Yes,Yes,Yes,Yes,Yes,No,Yes,NaN,Yes,Yes
1,L5,c.1091_1093del,p.Lys364del,Yes,Wide,Long,Thin,No,No,No,...,Yes,No,Yes,Yes,Yes,No,Yes,NaN,No,NaN
2,L18,c.1091_1093del,p.Lys364del,Yes,Normal,"Broad, long",Normal,Yes,NaN,NaN,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN,Yes,Yes
3,L37,c.1091_1093del,p.Lys364del,Yes,Wide,Broad,Thick,Yes,Yes,No,...,Yes,yes,Yes,Yes,Yes,Yes,No,NaN,No,Yes
4,Y4,c.1091_1093del,p.Lys364del,Yes,Wide,Broad,Thin,Yes,No,Yes,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes
5,Y21,c.1091_1093del,p.Lys364del,Yes,Wide,Long,Thin,Yes,Yes,No,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,No,Yes
6,Y22,c.1091_1093del,p.Lys364del,Yes,Wide,Long,Thin,Yes,Yes,No,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,NaN
7,Y29,c.1091_1093del,p.Lys364del,Yes,Normal,"Broad, short",Thin,No,No,No,...,Yes,Yes,No,Yes,Yes,Yes,No,NaN,Yes,Yes
8,K2588,c.1096c>T,p.Arg366Cys,Yes,Flat,Long,Thin,Yes,No,No,...,Yes,Yes,No,Yes,Yes,No,No,NaN,No,No
9,K2426,c.1121G>A,p.Arg374Gln,Yes,Flat,Normal,Thin,Yes,No,No,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN,NaN,No


Some column names might include spaces in front or after, and a couple of columns are subheadings and only contain NaNs, so lets correct that:

In [5]:
df.columns = df.columns.str.strip()
df = df.dropna(axis=1, how='all')
df.head()

,"Patient ID""",Nucleotide change,Amino acid change,Feeding difficulties,Nasal bridge,Phitrum,Upper lip vermilion,Thick lower lip vermilion,High palate,Cleft palate,...,Microcephaly,Sparse scalp hair,hypertrichosis,Thick eyebrows,Long eyelashes,Ptosis,Short 5th finger,Short 5th toe,Prominent interphalangeal joints,Prominent distal phalanges
0,L43,c.1089G>T,p.Lys363Asn,Yes,Narrow,Normal,Thin,No,Yes,No,...,Yes,Yes,Yes,Yes,Yes,No,Yes,NaN,Yes,Yes
1,L5,c.1091_1093del,p.Lys364del,Yes,Wide,Long,Thin,No,No,No,...,Yes,No,Yes,Yes,Yes,No,Yes,NaN,No,NaN
2,L18,c.1091_1093del,p.Lys364del,Yes,Normal,"Broad, long",Normal,Yes,NaN,NaN,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN,Yes,Yes
3,L37,c.1091_1093del,p.Lys364del,Yes,Wide,Broad,Thick,Yes,Yes,No,...,Yes,yes,Yes,Yes,Yes,Yes,No,NaN,No,Yes
4,Y4,c.1091_1093del,p.Lys364del,Yes,Wide,Broad,Thin,Yes,No,Yes,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes


<h2>Column mappers</h2>

In [6]:
hpo_cr = parser.get_hpo_concept_recognizer()
generator = SimpleColumnMapperGenerator(df=df, observed='Yes', excluded='No', hpo_cr=hpo_cr)
column_mapper_d = generator.try_mapping_columns()
from IPython.display import display, HTML
display(HTML(generator.to_html()))

Result,Columns
Mapped,Feeding difficulties; Thick lower lip vermilion; High palate; Cleft palate; Scoliosis; Hernia; Hearing impairment; Visual impairment; Frequent infections; Hypotonia; Seizures; Absent speech; Small for gestational age; Birth length less than 3rd percentile; Primary microcephaly; Decreased body weight; Short stature; Microcephaly; Sparse scalp hair; hypertrichosis; Thick eyebrows; Long eyelashes; Ptosis; Short 5th finger; Short 5th toe; Prominent interphalangeal joints
Unmapped,"Patient ID""; Nucleotide change; Amino acid change; Nasal bridge; Phitrum; Upper lip vermilion; Patient ID #; Cardiovascular; Gastrointestinal; Genitouriry; Ophthalmological abnormalities; CNS structural abnormalities; Developmental delay/intellectual disability; Behavioral abnormalities; Age; Sex; Prominent distal phalanges"


In [7]:
severity_id = {'Se': 'Intellectual disability, severe',
                 'Mo': 'Intellectual disability, moderate',
               'Mi':'Intellectual disability, mild'}
idMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=severity_id)
#print(idMapper.preview_column(dft['Developmental delay/intellectual disability']))
column_mapper_d['Developmental delay/intellectual disability'] = idMapper

In [8]:
nasal_bridge = {'Narrow': 'Narrow nasal bridge',
                 'Wide': 'Wide nasal bridge',
               'Flat':'Depressed nasal bridge'}
nasalMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=nasal_bridge)
#print(nasalMapper.preview_column(dft['Nasal bridge']))
column_mapper_d['Nasal bridge'] = idMapper

In [9]:
philtrum = {'Broad': 'Broad philtrum',
                 'Long': 'Long philtrum',
               'Flat':'Depressed nasal bridge',
            'Short':'Short philtrum'}
philtrumMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=philtrum )
#print(philtrumMapper.preview_column(dft['Phitrum']))
column_mapper_d['Phitrum'] = philtrumMapper

In [10]:
upperlip = {'Thin': 'Thin upper lip vermilion',
                 'Thick': 'Thick upper lip vermilion'}
upperlipMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=upperlip)
# print(upperlipMapper.preview_column(dft['Upper lip vermilion']))
column_mapper_d['Upper lip vermilion'] = upperlipMapper

In [11]:
cardiovascular = {'dex': 'Dextrocardia',
                 'ps': 'Pulmonic stenosis',
                 'vsd': 'Ventricular septal defect',
                 'asd': 'Atrial septal defect'}
cardiovascularMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=cardiovascular)
# print(cardiovascularMapper.preview_column(dft['Cardiovascular']))
column_mapper_d['Cardiovascular'] = cardiovascularMapper

In [12]:
gastrointestinal = {'pys': 'Pyloric stenosis',
                 'ps': 'Pulmonic stenosis',
                 'ger': 'Gastroesophageal reflux'}
gastrointestinalMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=gastrointestinal)
# print(gastrointestinalMapper.preview_column(dft['Gastrointestinal']))
column_mapper_d['Gastrointestinal'] = gastrointestinalMapper

In [13]:
genitouriry = {'hk': 'Horseshoe kidney',
                 'cr': 'cryptorchidism',
                 'VUR': 'Vesicoureteral reflux',
              'HN': 'hydronephrosis',
              'HU': 'Hydroureter',
              'VD': 'Urethral diverticulum'}
genitouriryMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=genitouriry)
# print(genitouriryMapper.preview_column(dft['Genitouriry']))
column_mapper_d['Genitouriry'] = genitouriryMapper

In [14]:
hernia = {'h': 'Hiatus hernia',
                 'u': 'umbilical hernia',
                 'i': 'inguinal hernia',
         'd': 'Congenital diaphragmatic hernia'}
herniaMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=hernia)
# print(herniaMapper.preview_column(dft['Hernia']))
column_mapper_d['Hernia'] = herniaMapper

In [15]:
opthal = {'my': 'Myopia',
                 'sph': 'spherophakia',
                 'am': 'amblyopia'}
opthalMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=opthal)
# print(opthalMapper.preview_column(dft['Ophthalmological abnormalities']))
column_mapper_d['Ophthalmological abnormalities'] = opthalMapper

In [16]:
corpus_callosum = {'acc': 'Abnormal corpus callosum morphology',
                 'ch': 'Aplasia/Hypoplasia of the cerebellum',
                 'dw': 'Dandy-Walker malformation'}
corpus_callosumMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=corpus_callosum)
# print(corpus_callosumMapper.preview_column(dft['CNS structural abnormalities']))
column_mapper_d['CNS structural abnormalities'] = corpus_callosumMapper

In [17]:
behavioral = {'HyAc': 'Hyperactivity',
                 'im': 'Impulsivity',
                 'tan': 'Abnormal temper tantrums'}
behavioralMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=behavioral)
# print(behavioralMapper.preview_column(dft['Behavioral abnormalities']))
column_mapper_d['Behavioral abnormalities'] = behavioralMapper

<h2>Variant Data</h2>
<p>The variant data (HGVS< transcript) is listed in the Variant (hg19, NM_015133.4) column.</p>

In [18]:
hg38 = 'hg38'
default_genotype = 'heterozygous'
SMARCB1_transcript='NM_003073.5'
# NM_003073.5(SMARCB1)
vvalidator = VariantValidator(genome_build=hg38, transcript=SMARCB1_transcript)
variant_d = {}
vlist = df['Nucleotide change'].unique()
for v in vlist:
    var = vvalidator.encode_hgvs(v)
    variant_d[v] = var

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003073.5%3Ac.1089G>T/NM_003073.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003073.5%3Ac.1091_1093del/NM_003073.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003073.5%3Ac.1096c>T/NM_003073.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003073.5%3Ac.1121G>A/NM_003073.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003073.5%3Ac.1130G>A/NM_003073.5?content-type=application%2Fjson


In [19]:
varMapper = VariantColumnMapper(variant_d=variant_d,
                                variant_column_name='Nucleotide change', 
                                default_genotype=default_genotype)

<h1>Demographic data</h1>

In [20]:
#age is in years and months, so manually correct it
ageMapper = AgeColumnMapper.by_year('Age')
ageMapper.preview_column(df['Age'])

,original column contents,age
0,13,P13Y
1,6,P6Y
2,9,P9Y
3,10,P10Y
4,21,P21Y
5,3,P3Y
6,1,P1Y
7,4,P4Y
8,7,P7Y


In [21]:
#sex is not in columns, since it were all females in this paper
sexMapper = SexColumnMapper(male_symbol='M', female_symbol='F', column_name='Sex')
sexMapper.preview_column(df['Sex'])

,original column contents,sex
0,F,FEMALE
1,F,FEMALE
2,F,FEMALE
3,M,MALE
4,F,FEMALE
5,F,FEMALE
6,M,MALE
7,M,MALE
8,M,MALE
9,M,MALE


In [22]:
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_mapper_d, 
                        individual_column_name='Patient ID"', 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=varMapper, 
                        metadata=metadata,
                        pmid=PMID)
encoder.set_disease(disease_id='OMIM:614608', label='Coffin-Siris syndrome 3 ')
individuals = encoder.get_individuals()

In [23]:
phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))

In [24]:
i1 = individuals[0]
phenopacket1 = i1.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh())
json_string = MessageToJson(phenopacket1)
print(json_string)

{
  "id": "PMID_25168959_L43",
  "subject": {
    "id": "L43",
    "timeAtLastEncounter": {
      "age": {
        "iso8601duration": "P13Y"
      }
    },
    "sex": "FEMALE"
  },
  "phenotypicFeatures": [
    {
      "type": {
        "id": "HP:0011968",
        "label": "Feeding difficulties"
      }
    },
    {
      "type": {
        "id": "HP:0000179",
        "label": "Thick lower lip vermilion"
      },
      "excluded": true
    },
    {
      "type": {
        "id": "HP:0000218",
        "label": "High palate"
      }
    },
    {
      "type": {
        "id": "HP:0000175",
        "label": "Cleft palate"
      },
      "excluded": true
    },
    {
      "type": {
        "id": "HP:0002650",
        "label": "Scoliosis"
      }
    },
    {
      "type": {
        "id": "HP:0000365",
        "label": "Hearing impairment"
      }
    },
    {
      "type": {
        "id": "HP:0000505",
        "label": "Visual impairment"
      },
      "excluded": true
    },
    {
      "t

In [25]:
output_directory = "phenopackets"
Individual.output_individuals_as_phenopackets(individual_list=individuals,
                                              pmid=PMID,
                                              metadata=metadata.to_ga4gh(),
                                              outdir=output_directory)

We output 11 GA4GH phenopackets to the directory phenopackets
